## Analysis of Monte-Carlo samples.

Here we perform Monte-Carlo analysis of the resampled images to study numerical stability of the tested reconstruction methods and statistical errors.

To obtain resampled images, the following procedure should be followed:
* run `resample_images.ipynb` notebook to generate resampled images from the original raw camera images.
* Run `process-resampled-cnn.py` and `process-resampled-rl.py` to reconstruct the images using CFCNN and RL.
* Results from ThunderStorm are bundled here. Otherwise, one would have to convert the resampled images to tiff and process them in ImageJ in bulk. A clue how to automate that is in `resolution-test/generate_inputs.py` in the commented section.
* Once the data are ready, one can run this notebook.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import h5py
from scipy.stats import entropy
from IPython.display import Markdown, display


def norm(a : np.ndarray) -> np.ndarray:
    """Normalize image by subtracting its minimum and setting L1 norm to 1."""
    asu = a - np.min(a)
    norm = np.sum(asu)
    if norm == 0:
        return np.zeros_like(a)
    return asu/norm

def MAE(imA : np.ndarray, imB : np.ndarray, blur : float =2) -> float:
    """
    Mean absolute error between two images. First, images are convolved with Gaussian kernel
    Args:
        imA, imB ... images
        blur ... sigma of gaussinan kernel (see scipy.ndimage.gaussian_filter)
    Returns:
        mean absolute error after blurring and normalization
    """
    imAn = norm(ndi.gaussian_filter(imA, blur))
    imBn = norm(ndi.gaussian_filter(imB, blur))
    return np.mean(np.abs(imAn.ravel() - imBn.ravel()))

def KLD(tested_image : np.ndarray, reference_image : np.ndarray, blur : float =2, prevent_infty : bool =True) -> float:
    """
    Kullback-Leibler divergence between two images after blurring and normalization.
    Args:
        tested_image ... images
        reference_image ... translates to reference distribution
        blur (default 1) ... sigma of gaussinan kernel (see scipy.ndimage.gaussian_filter)
        prevent_infinity (default True) ... if true, a small epsilon (1e-16) is added to image to avoid zeros (infs in logs) in the tested image.
    Returns:
        Kullback-Leibner divergence, in bits (we use log2 in entropy formula).
    """    
    # + 1e-9 #tested distribution
    imAn = norm(ndi.gaussian_filter(tested_image, blur).ravel())
    if prevent_infty:
        imAn += 1e-16
        imAn /= np.sum(imAn)
    # + 1e-9 #reference distribution
    imBn = norm(ndi.gaussian_filter(reference_image, blur).ravel())
    return entropy(imBn/np.sum(imBn), imAn/np.sum(imAn), base=2)

#Dictionaries of metrics to be evaluated
metrics = {'MAE': MAE, 'KLD': KLD}

In [2]:
datasets = (1, 2, 3)
h5_files = {i: f'../Image{i}.h5' for i in datasets}
h5_files_resampled_cnn = {i: f'mc_img{i}_cnn.h5' for i in datasets}
h5_files_resampled_rl = {i: f'mc_img{i}_rl.h5' for i in datasets}
h5_files_resampled_mef = {i: f'mc_img{i}_mef.h5' for i in datasets}

keys_template = ['img{n}_camera_image', 'img{n}_R-L',
                 'img{n}_MEF', 'img{n}_CFCNN', 'img{n}_ground_truth']

references = dict()

results = dict()
#These quantiles are selected in accordance to normal distribution as an equivalent to standard deviation in
#case of symmetrical distribution.
#But it is better way to characterize variability for asymetric distribution.
LO_QUANTILE = 0.159
HI_QUANTILE = 0.841
METHODS = ['RL', 'MEF', 'CFCNN']

#iterate overl all presented images
for dset_idx in datasets:
    #original images
    with h5py.File(h5_files[dset_idx], 'r') as h5f:
        reference = np.array(h5f[f'img{dset_idx}_ground_truth'])
        rl_img = np.array(h5f[f'img{dset_idx}_R-L'])
        cnn_img = np.array(h5f[f'img{dset_idx}_CFCNN'])
        mef_img = np.array(h5f[f'img{dset_idx}_MEF'])
    #load outputs of various reconstruction methods corresponding to 
    #resampled inputs
    with h5py.File(h5_files_resampled_cnn[dset_idx], 'r') as h5f:
        mc_cnn_img = np.array(h5f['resampled'])[:, :, :, 0]
    with h5py.File(h5_files_resampled_rl[dset_idx], 'r') as h5f:
        mc_rl_img = np.array(h5f['resampled'])
    with h5py.File(h5_files_resampled_mef[dset_idx], 'r') as h5f:
        _key = list(h5f.keys())[0]
        mc_mef_img = np.array(h5f[_key])

    #Assembly all metrics of interest and write them in a big dictionary.
    for mkey, metric in metrics.items():
        results[f'Image{dset_idx}/RL/{mkey}'] = metric(rl_img, reference)
        results[f'Image{dset_idx}/CFCNN/{mkey}'] = metric(cnn_img, reference)
        results[f'Image{dset_idx}/MEF/{mkey}'] = metric(mef_img, reference)

        for method, dset in zip(METHODS, (mc_rl_img, mc_mef_img, mc_cnn_img)):
            results[f'Image{dset_idx}/{method}/{mkey}_std'] = np.std(
                [metric(_img, reference) for _img in dset])
            results[f'Image{dset_idx}/{method}/{mkey}_lo'] = np.quantile(
                [metric(_img, reference) for _img in dset], LO_QUANTILE)            
            results[f'Image{dset_idx}/{method}/{mkey}_hi'] = np.quantile(
                [metric(_img, reference) for _img in dset], HI_QUANTILE)
            
for key in sorted(results.keys()):
    print(f'{key} : {results[key]:.3e}')

Image1/CFCNN/KLD : 1.080e+00
Image1/CFCNN/KLD_hi : 4.753e+00
Image1/CFCNN/KLD_lo : 1.384e+00
Image1/CFCNN/KLD_std : 1.558e+00
Image1/CFCNN/MAE : 2.412e-05
Image1/CFCNN/MAE_hi : 3.798e-05
Image1/CFCNN/MAE_lo : 2.587e-05
Image1/CFCNN/MAE_std : 5.497e-06
Image1/MEF/KLD : 1.260e+01
Image1/MEF/KLD_hi : 4.551e+01
Image1/MEF/KLD_lo : 4.551e+01
Image1/MEF/KLD_std : 1.099e-07
Image1/MEF/MAE : 3.987e-05
Image1/MEF/MAE_hi : 5.000e-05
Image1/MEF/MAE_lo : 5.000e-05
Image1/MEF/MAE_std : 1.500e-12
Image1/RL/KLD : 5.812e+00
Image1/RL/KLD_hi : 1.014e+01
Image1/RL/KLD_lo : 6.542e+00
Image1/RL/KLD_std : 1.738e+00
Image1/RL/MAE : 4.640e-05
Image1/RL/MAE_hi : 4.797e-05
Image1/RL/MAE_lo : 4.633e-05
Image1/RL/MAE_std : 7.859e-07
Image2/CFCNN/KLD : 2.096e+00
Image2/CFCNN/KLD_hi : 2.864e+00
Image2/CFCNN/KLD_lo : 2.263e+00
Image2/CFCNN/KLD_std : 3.632e-01
Image2/CFCNN/MAE : 2.269e-05
Image2/CFCNN/MAE_hi : 2.397e-05
Image2/CFCNN/MAE_lo : 2.240e-05
Image2/CFCNN/MAE_std : 7.776e-07
Image2/MEF/KLD : 1.727e+01
Image

In [4]:
# Present table with metrics

# Helper function to round values based on the standard deviation
def round_with_std(value : float, std : float) -> str:
    # Determine precision based on std
    precision = -int(np.floor(np.log10(std)))
    return f"{value:.{precision}f} ± {std:.{precision}f}"


metric_scales = {'KLD': 1, 'MAE': 1e5}
metric_scales_labels = {'KLD': '', 'MAE': '[×10⁻⁵]'}

table = "| Dataset | Metric | R-L | MEF | CFCNN |\n"
table += "|--------|--------|-----|-----|-------|\n"

for dset_idx in datasets:
    for mkey in metrics:
        scale = metric_scales[mkey]
        scale_label = metric_scales_labels[mkey]
        line_numbers = []
        for method in METHODS:
            mean = results[f'Image{dset_idx}/{method}/{mkey}']*scale
            std = results[f'Image{dset_idx}/{method}/{mkey}_std']*scale
            # print(mean, std)
            line_numbers.append(round_with_std(mean, std))
        number_content = '| '.join(line_numbers)
        table += f'| Image{dset_idx} | {mkey}{scale_label} | {number_content} |\n'

display(Markdown(table))

| Dataset | Metric | R-L | MEF | CFCNN |
|--------|--------|-----|-----|-------|
| Image1 | MAE[×10⁻⁵] | 4.64 ± 0.08| 3.9872019 ± 0.0000002| 2.4 ± 0.5 |
| Image1 | KLD | 6 ± 2| 12.5980058 ± 0.0000001| 1 ± 2 |
| Image2 | MAE[×10⁻⁵] | 3.98 ± 0.07| 3.6 ± 0.1| 2.27 ± 0.08 |
| Image2 | KLD | 7.8 ± 0.7| 17 ± 2| 2.1 ± 0.4 |
| Image3 | MAE[×10⁻⁵] | 3.52 ± 0.03| 4.01 ± 0.02| 3.34 ± 0.04 |
| Image3 | KLD | 10.0 ± 0.3| 19.0 ± 0.4| 5.2 ± 0.2 |


**Notes:**

* If errorbar exceeds the definition interval, it is because the variance is quite high (and assymetric) with respect to the measured original value. The original value was lower then the mean value as asymetric resampled distribution.

* The extremely low errors for MEF analysis of Image1 is due to the fact, that most of the time, the MEF fitting yield no emitters, and therefore these results were discarded. In the small remaining part of the images, the results were numerically stable.